In [0]:
#!unzip factRuEval-2016-master.zip
#!pip install deep_ner

In [5]:
from deep_ner.elmo_ner import ELMo_NER
from deep_ner.utils import load_dataset_from_json, factrueval2016_to_json
import json
import os

In [6]:
%%time
factrueval2016_to_json("factRuEval-2016-master/devset", "factrueval2016devset_to_json.json")
X, y = load_dataset_from_json("factrueval2016devset_to_json.json")

recognizer = ELMo_NER(
    
            finetune_elmo=False, 
            batch_size=16, 
            l2_reg=1e-2, 
            max_seq_length=200,
            elmo_hub_module_handle='http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz', 
            validation_fraction=0.25, 
            max_epochs=1,
            patience=10, 
            gpu_memory_frac=0.9, 
            verbose=True, 
            random_seed=42, 
            lr=1e-2, 
            udpipe_lang='ru',
            use_additional_features=False

        )

recognizer.fit(X, y)

CPU times: user 26min 26s, sys: 43.7 s, total: 27min 10s
Wall time: 14min 25s


In [0]:
#!mkdir FactRuEval2016_results
#!mkdir FactRuEval2016_results/results_of_elmo_and_crf

In [8]:
%%time
factrueval2016_to_json("factRuEval-2016-master/testset", "factrueval2016testset_to_json.json")

with open("factrueval2016testset_to_json.json", 'r') as fp:
  data_for_testing = json.load(fp)

_, true_entities = load_dataset_from_json("factrueval2016testset_to_json.json")

texts = []
additional_info = []
for cur_document in data_for_testing:
  base_name = os.path.join("FactRuEval2016_results/results_of_elmo_and_crf", cur_document['base_name'] + '.task1')
  for cur_paragraph in cur_document['paragraph_bounds']:
    texts.append(cur_document['text'][cur_paragraph[0]:cur_paragraph[1]])
    additional_info.append((base_name, cur_paragraph))

predicted_entities = recognizer.predict(texts)

results_for_factrueval_2016 = dict()
for sample_idx, cur_result in enumerate(predicted_entities):
    base_name, paragraph_bounds = additional_info[sample_idx]
    for entity_type in cur_result:
        if entity_type == 'ORG':
            prepared_entity_type = 'org'
        elif entity_type == 'PERSON':
            prepared_entity_type = 'per'
        elif entity_type == 'LOCATION':
            prepared_entity_type = 'loc'
        else:
            prepared_entity_type = None
        
        for entity_bounds in cur_result[entity_type]:
            postprocessed_entity = (
                prepared_entity_type,
                entity_bounds[0] + paragraph_bounds[0],
                entity_bounds[1] - entity_bounds[0]
                )
            if base_name in results_for_factrueval_2016:
                results_for_factrueval_2016[base_name].append(postprocessed_entity)
            else:
                results_for_factrueval_2016[base_name] = [postprocessed_entity]

for base_name in results_for_factrueval_2016:
    with open(base_name, 'w+') as fp:
        for cur_entity in sorted(results_for_factrueval_2016[base_name], key=lambda it: (it[1], it[2], it[0])):
            fp.write('{0} {1} {2}\n'.format(cur_entity[0], cur_entity[1], cur_entity[2]))

CPU times: user 34min 42s, sys: 35.4 s, total: 35min 18s
Wall time: 18min 32s


In [0]:
#!python factRuEval-2016-master/scripts/t1_eval.py -s /content/factRuEval-2016-master -t /content/FactRuEval2016_results/results_of_elmo_and_crf

In [0]:
#!zip -r /content/file.zip /content/FactRuEval2016_results